In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()
spark

In [17]:
#### Requirement 4

from pyspark.sql import Row
from pyspark.sql.functions import col,array_contains
from pyspark.sql.types import ArrayType,DoubleType,BooleanType
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
salesDF=spark.read.option("header",True).csv("C:/Users/gomall/Downloads/sparsales.csv")
salesDF1=salesDF.coalesce(1)
resultDF= salesDF1.filter(col("Item Type").like("%a%a%")).select("Item Type").distinct()
resultDF.show()
resultDF.write.parquet("C:/Users/gomall/Desktop/requirement4pysaprk.parquet")



+-------------+
|    Item Type|
+-------------+
|Personal Care|
+-------------+



In [ ]:
## requirement5 

from pyspark.sql import Row
from pyspark.sql.functions import col,array_contains
from pyspark.sql.types import ArrayType,DoubleType,BooleanType
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank, dense_rank, avg
salesDF=spark.read.option("header",True).csv("C:/Users/gomall/Downloads/sparsales.csv")
# 3. Aggregate total units sold per region & country
salesDF1=salesDF.coalesce(1)
agg_df = salesDF1.groupBy("Region", "Country").agg(sum("Units Sold").alias("TotalUnitsSold"))
 
# 4. Define a window partitioned by region, ordered by TotalUnitsSold desc
window_spec = Window.partitionBy("Region").orderBy(col("TotalUnitsSold").desc())
 
# 5. Add rank column
ranked_df = agg_df.withColumn("rank",row_number().over(window_spec))
 
# 6. Filter to keep only the top-ranked country in each region
result_df = ranked_df.filter(col("rank") == 1).select("Region", "Country", "TotalUnitsSold")
result_df.show()
result_df.write.parquet("C:/Users/gomall/Desktop/requirement5pysaprk.parquet")